# Grupo 6
## Integrantes 
- Pablo Sarzosa
- Elisa Herrera
- Doménica Rodríguez 
- Andrés Albarracín

# 8.5. Implementación de Redes Neuronales Recurrentes desde Cero

En esta sección implementaremos un RNN desde cero para un modelo de lenguaje a nivel de caracteres, de acuerdo con nuestras descripciones en la Sección 8.4 . Tal modelo será entrenado en La máquina del tiempo de HG Wells . Como antes, comenzamos leyendo primero el conjunto de datos, que se presenta en la Sección 8.3 .

In [1]:
use strict;
use warnings;
use Data::Dump qw(dump);
use Data::Dumper qw(Dumper);
use AI::MXNet qw(mx);
use List::Util qw(max min shuffle);
use d2l;
use d2l::Timer;
use d2l::Animator;
use d2l::Accumulator;
IPerl->load_plugin('Chart::Plotly');
use Chart::Plotly::Plot;
use constant is_train =>'False';


In [2]:
use strict;
use warnings;


my ($batch_size, $num_steps) = (32, 35);
my ($train_iter, $vocab) = d2l::load_data_time_machine($batch_size, $num_steps);


Error: Can't locate object method "new" via package "SeqDataLoader" (perhaps you forgot to load "SeqDataLoader"?) at /usr/local/lib/perl5/site_perl/5.32.1/x86_64-linux/d2l.pm line 887.


## 8.5.1. Codificación One-Hot
Recuerde que cada token se representa como un índice numérico en train_iter. Alimentar estos índices directamente a una red neuronal puede dificultar el aprendizaje. A menudo representamos cada token como un vector de características más expresivo. La representación más fácil se denomina codificación one-hot , que se presenta en la Sección 3.4.1 .

En pocas palabras, asignamos cada índice a un vector unitario diferente: supongamos que el número de tokens diferentes en el vocabulario es N  ( len(vocab)) y los índices de los tokens van de 0 a N−1 . Si el índice de un token es el número entero i , luego creamos un vector de todos los 0 con una longitud de N y establecer el elemento en la posición  i a 1. Este vector es el vector caliente del token original. Los vectores one-hot con índices 0 y 2 se muestran a continuación.

In [3]:


my @indices = (0, 2);
my $vocab_size = scalar(@vocab);

my @one_hot = map { [(0) x $vocab_size] } @indices;

foreach my $i (@indices) {
  $one_hot[$i][$i] = 1;
}

print "@$_\n" foreach @one_hot;


Error: Global symbol "@vocab" requires explicit package name (did you forget to declare "my @vocab"?) at reply input line 4.
BEGIN not safe after errors--compilation aborted at reply input line 17.



La forma del mini lote que muestreamos cada vez es (tamaño del lote, número de pasos de tiempo). La one_hotfunción transforma dicho minilote en un tensor tridimensional con la última dimensión igual al tamaño del vocabulario ( len(vocab)). A menudo transponemos la entrada para obtener una salida de forma (número de pasos de tiempo, tamaño del lote, tamaño del vocabulario). Esto nos permitirá recorrer de manera más conveniente la dimensión más externa para actualizar los estados ocultos de un minilote, paso a paso.

In [4]:

use PDL; # Módulo para operaciones numéricas similares a NumPy

my $X = pdl [0..9]->reshape(2, 5);
my $one_hot = pdl [0..27]->reshape(28, 1);
my $result = $one_hot->index($X->transpose)->transpose->reshape($X->dim(0), $X->dim(1), 28);

print Dumper($result->shape);


Warning: Prototype mismatch: sub main::max (@) vs none at (eval 1646) line 7.

Prototype mismatch: sub main::min (@) vs none at (eval 1646) line 7.


Error: Can't call method "reshape" on unblessed reference at reply input line 4.


## 8.5.2. Inicialización de los parámetros del modelo
A continuación, inicializamos los parámetros del modelo para el modelo RNN. El número de unidades ocultas num_hiddenses un hiperparámetro ajustable. Al entrenar modelos de lenguaje, las entradas y salidas son del mismo vocabulario. Por lo tanto, tienen la misma dimensión, que es igual al tamaño del vocabulario.

In [5]:
use PDL; # Módulo para operaciones numéricas similares a NumPy

sub normal {
    my ($shape, $device) = @_;
    return (pdl zeroes $shape)->grandom->mul(0.01)->astype('float32')->to($device);
}

sub get_params {
    my ($vocab_size, $num_hiddens, $device) = @_;

    my ($num_inputs, $num_outputs) = ($vocab_size, $vocab_size);

    # Hidden layer parameters
    my $W_xh = normal([$num_inputs, $num_hiddens], $device);
    my $W_hh = normal([$num_hiddens, $num_hiddens], $device);
    my $b_h = zeroes($num_hiddens)->astype('float32')->to($device);
    # Output layer parameters
    my $W_hq = normal([$num_hiddens, $num_outputs], $device);
    my $b_q = zeroes($num_outputs)->astype('float32')->to($device);
    # Attach gradients
    my @params = ($W_xh, $W_hh, $b_h, $W_hq, $b_q);
    foreach my $param (@params) {
        $param->inplace->zero_grad;
    }
    return @params;
}

my $vocab_size = 100;
my $num_hiddens = 256;
my $device = pdl; # Use the CPU by default
my @params = get_params($vocab_size, $num_hiddens, $device);

# Imprimir la forma de los parámetros
print $_->shape, "\n" foreach @params;


Error: Can't call method "initialize" on unblessed reference at /usr/local/lib/perl5/site_perl/5.32.1/x86_64-linux/PDL/Core.pm line 2383.


## 8.5.3. Modelo RNN
Para definir un modelo RNN, primero necesitamos una init_rnn_statefunción para devolver el estado oculto en la inicialización. Devuelve un tensor lleno de 0 y con forma de (tamaño de lote, número de unidades ocultas). El uso de tuplas facilita el manejo de situaciones en las que el estado oculto contiene múltiples variables, que encontraremos en secciones posteriores.

In [6]:

use PDL; # Módulo para operaciones numéricas similares a NumPy

sub init_rnn_state {
    my ($batch_size, $num_hiddens, $device) = @_;
    my $h = zeroes([$batch_size, $num_hiddens])->astype('float32')->to($device);
    return ($h);
}

my $batch_size = 32;
my $num_hiddens = 256;
my $device = pdl; # Use the CPU by default
my ($h) = init_rnn_state($batch_size, $num_hiddens, $device);

# Imprimir la forma del estado oculto
print $h->shape, "\n";

Error: Can't call method "initialize" on unblessed reference at /usr/local/lib/perl5/site_perl/5.32.1/x86_64-linux/PDL/Core.pm line 2383.


La siguiente rnnfunción define cómo calcular el estado oculto y la salida en un paso de tiempo. Tenga en cuenta que el modelo RNN recorre la dimensión más externa de inputsmodo que actualiza los estados ocultos H de un minilote, paso a paso. Además, la función de activación aquí utiliza el tanh función. Como se describe en la Sección 4.1 , el valor medio de la tanh función es 0, cuando los elementos se distribuyen uniformemente sobre los números reales.

In [7]:

use PDL; # Módulo para operaciones numéricas similares a NumPy

sub rnn {
    my ($inputs, $state, $params) = @_;
    my ($W_xh, $W_hh, $b_h, $W_hq, $b_q) = @{$params};
    my ($H) = @{$state};
    my @outputs;
    for my $X (transpose($inputs, [1, 0, 2])->dog) {
        $H = tanh($X x $W_xh + $H x $W_hh + $b_h);
        my $Y = $H x $W_hq + $b_q;
        push @outputs, $Y;
    }
    my $concatenated_outputs = cat(@outputs)->dog;
    return ($concatenated_outputs, [$H]);
}

my $batch_size = 32;
my $num_steps = 35;
my $vocab_size = 10000;
my $num_hiddens = 256;
my $inputs = zeroes([$num_steps, $batch_size, $vocab_size])->astype('float32');
my $state = zeroes([$batch_size, $num_hiddens])->astype('float32');
my $params = [map { zeroes([@$_])->astype('float32') } ($vocab_size, $num_hiddens, $num_hiddens, $num_hiddens, $vocab_size)];
my ($outputs, $new_state) = rnn($inputs, [$state], $params);

# Imprimir la forma de las salidas y del nuevo estado oculto
print $outputs->shape, "\n"; # Salida: [1120, 10000]
print $new_state->[0]->shape, "\n"; # Salida: [32, 256]


Error: Can't call method "initialize" on unblessed reference at /usr/local/lib/perl5/site_perl/5.32.1/x86_64-linux/PDL/Core.pm line 2383.


Con todas las funciones necesarias definidas, a continuación creamos una clase para envolver estas funciones y almacenar parámetros para un modelo RNN implementado desde cero.

In [8]:
package RNNModelScratch;
use strict;
use warnings;
use v5.26;
use PDL;
use PDL::NiceSlice;

sub new {
    my ($class, $vocab_size, $num_hiddens, $device, $get_params, $init_state, $forward_fn) = @_;
    my $self = bless {}, $class;
    $self->{vocab_size} = $vocab_size;
    $self->{num_hiddens} = $num_hiddens;
    $self->{params} = $get_params->($vocab_size, $num_hiddens, $device);
    $self->{init_state} = $init_state;
    $self->{forward_fn} = $forward_fn;
    return $self;
}

sub begin_state {
    my ($self, $batch_size, $ctx) = @_;
    return $self->{init_state}->($batch_size, $self->{num_hiddens}, $ctx);
}

sub __call__ {
    my ($self, $X, $state) = @_;
    $X = one_hot($X->transpose(), $self->{vocab_size});
    return $self->{forward_fn}->($X, $state, $self->{params});
}

1;


1

Verifiquemos si las salidas tienen las formas correctas, por ejemplo, para garantizar que la dimensionalidad del estado oculto permanezca sin cambios.

In [9]:
my $num_hiddens = 512;
my $net = RNNModelScratch->new(len($vocab), $num_hiddens, d2l->try_gpu(), \&get_params,
                               \&init_rnn_state, \&rnn);
my $state = $net->begin_state($X->shape->[0], d2l->try_gpu());
my ($Y, $new_state) = $net->($X->as_in_context(d2l->try_gpu()), $state);
say $Y->shape, scalar @$new_state, $new_state->[0]->shape;


Error: Undefined subroutine &RNNModelScratch::len called at reply input line 2.


Podemos ver que la forma de salida es (número de pasos de tiempo ×  tamaño del lote, tamaño del vocabulario), mientras que la forma del estado oculto sigue siendo la misma, es decir, (tamaño del lote, número de unidades ocultas).

## 8.5.4. Predicción
Primero definamos la función de predicción para generar nuevos caracteres siguiendo el proporcionado por el usuario prefix, que es una cadena que contiene varios caracteres. Al recorrer estos caracteres iniciales en prefix, seguimos pasando el estado oculto al siguiente paso de tiempo sin generar ningún resultado. Esto se denomina período de calentamiento , durante el cual el modelo se actualiza (por ejemplo, actualiza el estado oculto) pero no hace predicciones. Después del período de calentamiento, el estado oculto es generalmente mejor que su valor inicializado al principio. Entonces generamos los caracteres predichos y los emitimos.

In [10]:
sub predict_ch8 {
    my ($prefix, $num_preds, $net, $vocab, $device) = @_;

    my $state = $net->begin_state(batch_size => 1, ctx => $device);
    my @outputs = ($vocab->{$prefix->[0]});
    my $get_input = sub {
        my $out = [ $outputs[-1] ];
        return np->array($out, ctx => $device)->reshape((1, 1));
    };
    for my $y (@{ $prefix }[1..$#{ $prefix }]) {  # Warm-up period
        my ($_, $state) = $net->($get_input->(), $state);
        push @outputs, $vocab->{$y};
    }
    for (1..$num_preds) {  # Predict `num_preds` steps
        my ($y, $state) = $net->($get_input->(), $state);
        push @outputs, int($y->argmax(axis => 1)->reshape(1)->at(0));
    }
    return join('', map { $vocab->idx_to_token->[$_]; } @outputs);
}


Ahora podemos probar la predict_ch8función. Especificamos el prefijo como y hacemos que genere 10 caracteres adicionales. Dado que no hemos entrenado la red, generará predicciones sin sentido.time traveller

In [11]:
predict_ch8('time traveller ', 10, net, vocab, d2l::try_gpu())

Error: Bareword "net" not allowed while "strict subs" in use at reply input line 1.
Bareword "vocab" not allowed while "strict subs" in use at reply input line 1.



## 8.5.5. Recorte de degradado
Para una secuencia de longitud T , calculamos los gradientes sobre estos  T pasos de tiempo en una iteración, lo que da como resultado una cadena de productos de matriz con longitud O(T) durante la retropropagación. Como se mencionó en la Sección 4.8 , podría resultar en inestabilidad numérica, por ejemplo, los gradientes pueden explotar o desaparecer, cuando T es largo. Por lo tanto, los modelos RNN a menudo necesitan ayuda adicional para estabilizar el entrenamiento.

En términos generales, al resolver un problema de optimización, tomamos medidas de actualización para el parámetro del modelo, por ejemplo, en forma de vector.  x , en la dirección del gradiente negativo  g en un mini lote. Por ejemplo, con η>0 como la tasa de aprendizaje, en una iteración actualizamos x como  x−ηg . Supongamos además que la función objetivo f se comporta bien, digamos, Lipschitz continuo con constante L . Es decir, para cualquier  x y y tenemos

(8.5.1)
|f(x)−f(y)|≤L∥x−y∥.
 
En este caso, podemos asumir con seguridad que si actualizamos el vector de parámetros por ηg , entonces

(8.5.2)
|f(x)−f(x−ηg)|≤Lη∥g∥,
 
lo que significa que no observaremos un cambio por más de  Lη∥g∥ . Esto es tanto una maldición como una bendición. Por el lado de la maldición, limita la velocidad de progreso; mientras que en el lado de la bendición, limita la medida en que las cosas pueden salir mal si nos movemos en la dirección equivocada.

A veces, los gradientes pueden ser bastante grandes y es posible que el algoritmo de optimización no converja. Podríamos abordar esto reduciendo la tasa de aprendizaje. η . Pero, ¿y si rara vez obtenemos grandes gradientes? En este caso, tal enfoque puede parecer totalmente injustificado. Una alternativa popular es recortar el degradado  g proyectándolos de nuevo a una bola de un radio dado, digamos θ a través de

(8.5.3)
g←min(1,θ∥g∥)g.
 
Al hacerlo, sabemos que la norma del gradiente nunca excede θ  y que el gradiente actualizado está completamente alineado con la dirección original de g . También tiene el efecto secundario deseable de limitar la influencia que cualquier minilote dado (y dentro de él, cualquier muestra dada) puede ejercer sobre el vector de parámetros. Esto otorga un cierto grado de robustez al modelo. El recorte de degradado proporciona una solución rápida a la explosión del degradado. Si bien no resuelve completamente el problema, es una de las muchas técnicas para aliviarlo.

A continuación, definimos una función para recortar los gradientes de un modelo que se implementa desde cero o un modelo construido por las API de alto nivel. También tenga en cuenta que calculamos la norma de gradiente sobre todos los parámetros del modelo.

In [12]:
sub grad_clipping {
    my ($net, $theta) = @_;
    my @params;
    if ($net->isa("AI::MXNet::Gluon::Block")) {
        @params = map { $_->data } $net->collect_params->values;
    } else {
        @params = @{$net->params};
    }
    my $norm = sqrt(sum(map { ($_->grad**2)->sum } @params));
    if ($norm > $theta) {
        for my $param (@params) {
            $param->grad *= $theta / $norm;
        }
    }
}


## 8.5.6. Capacitación
Antes de entrenar el modelo, definamos una función para entrenar el modelo en una época. Difiere de cómo entrenamos el modelo de la Sección 3.6 en tres lugares:

Diferentes métodos de muestreo para datos secuenciales (muestreo aleatorio y partición secuencial) darán como resultado diferencias en la inicialización de estados ocultos.

Recortamos los gradientes antes de actualizar los parámetros del modelo. Esto asegura que el modelo no diverja incluso cuando los gradientes aumentan en algún punto durante el proceso de entrenamiento.

Utilizamos la perplejidad para evaluar el modelo. Como se discutió en la Sección 8.4.4 , esto asegura que las secuencias de diferente longitud sean comparables.

Específicamente, cuando se usa el particionamiento secuencial, inicializamos el estado oculto solo al comienzo de cada época. Desde el  ith ejemplo de subsecuencia en el siguiente minilote es adyacente al actual ith ejemplo de subsecuencia, el estado oculto al final del minilote actual se usará para inicializar el estado oculto al comienzo del siguiente minilote. De esta forma, la información histórica de la secuencia almacenada en estado oculto podría fluir sobre subsecuencias adyacentes dentro de una época. Sin embargo, el cálculo del estado oculto en cualquier punto depende de todos los minilotes anteriores en la misma época, lo que complica el cálculo del gradiente. Para reducir el costo computacional, separamos el gradiente antes de procesar cualquier minilote para que el cálculo del gradiente del estado oculto siempre se limite a los pasos de tiempo en un minilote.

Cuando usamos el muestreo aleatorio, necesitamos reinicializar el estado oculto para cada iteración ya que cada ejemplo se muestrea con una posición aleatoria. Igual que la train_epoch_ch3función en la Sección 3.6 , updateres una función general para actualizar los parámetros del modelo. Puede ser la d2l.sgdfunción implementada desde cero o la función de optimización integrada en un marco de aprendizaje profundo.

In [13]:
#@save
sub train_epoch_ch8 {
    my ($net, $train_iter, $loss, $updater, $device, $use_random_iter) = @_;
    my ($state, $timer) = (undef, d2l::Timer->new());
    my $metric = d2l::Accumulator->new(2); # Sum of training loss, no. of tokens

    while (my ($X, $Y) = $train_iter->()) {
        if (!defined $state || $use_random_iter) {
            # Initialize `state` when either it is the first iteration or
            # using random sampling
            $state = $net->begin_state(batch_size => $X->shape->[0], ctx => $device);
        } else {
            $_->detach for @$state;
        }
        my $y = $Y->T->reshape(-1);
        $X = $X->as_in_ctx($device);
        $y = $y->as_in_ctx($device);

        autograd->record(sub {
            my ($y_hat, $state) = $net->($X, $state);
            my $l = $loss->($y_hat, $y)->mean;
            $l->backward;
        });

        grad_clipping($net, 1);
        $updater->(batch_size => 1); # Since the `mean` function has been invoked
        $metric->add($l * $y->size, $y->size);
    }
    return exp($metric->[0] / $metric->[1]), $metric->[1] / $timer->stop;
}


Error: Global symbol "$l" requires explicit package name (did you forget to declare "my $l"?) at reply input line 27.
BEGIN not safe after errors--compilation aborted at reply input line 35.



La función de entrenamiento admite un modelo RNN implementado desde cero o utilizando API de alto nivel.

In [14]:
#@save
sub train_epoch_ch8 {
    my ($net, $train_iter, $loss, $updater, $device, $use_random_iter) = @_;

    my ($state, $timer) = (undef, d2l::Timer->new());
    my $metric = d2l::Accumulator->new(2);  # Sum of training loss, no. of tokens

    while (my ($X, $Y) = $train_iter->()) {
        if (!defined $state || $use_random_iter) {
            # Initialize `state` when either it is the first iteration or
            # using random sampling
            $state = $net->begin_state(
                batch_size => $X->shape->[0],
                ctx => $device,
            );
        } else {
            $_->detach() for @$state;
        }

        my $y = $Y->T->reshape(-1);
        ($X, $y) = ($X->as_in_ctx($device), $y->as_in_ctx($device));

        autograd->with(autograd->record(), sub {
            my ($y_hat, $state) = $net->($X, $state);
            my $l = $loss->($y_hat, $y)->mean();
            $l->backward();
            grad_clipping($net, 1);
            $updater->(batch_size => 1);  # Since the `mean` function has been invoked
            $metric->add($l * $y->size, $y->size);
        });
    }

    return (
        exp($metric->[0] / $metric->[1]),
        $metric->[1] / $timer->stop(),
    );
}

sub train_ch8 {
    my ($net, $train_iter, $vocab, $lr, $num_epochs, $device,
        $use_random_iter) = @_;

    my $loss = gluon->loss->SoftmaxCrossEntropyLoss();
    my $animator = d2l::Animator->new(
        xlabel => 'epoch',
        ylabel => 'perplexity',
        legend => ['train'],
        xlim => [10, $num_epochs],
    );

    # Initialize
    if ($net->isa('gluon::Block')) {
        $net->initialize(
            ctx => $device,
            force_reinit => 1,
            init => init->Normal(0.01),
        );
        my $trainer = gluon->Trainer(
            $net->collect_params(),
            'sgd',
            {learning_rate => $lr},
        );
        my $updater = sub {
            my ($batch_size) = @_;
            $trainer->step($batch_size);
        };
    } else {
        my $updater = sub {
            my ($batch_size) = @_;
            d2l->sgd($net->params, $lr, $batch_size);
        };
    }

    my $predict = sub { predict_ch8($_[0], 50, $net, $vocab, $device) };

    # Train and predict
    for my $epoch (0 .. $num_epochs - 1) {
        my ($ppl, $speed) = train_epoch_ch8(
            $net, $train_iter, $loss, $updater, $device, $use_random_iter,
        );

        if (($epoch + 1) % 10 == 0) {
            $animator->add($epoch + 1, [$ppl]);
        }
    }

    print "perplexity $ppl, $speed tokens


Error: Can't find string terminator '"' anywhere before EOF at reply input line 87.



Ahora podemos entrenar el modelo RNN. Dado que solo usamos 10000 tokens en el conjunto de datos, el modelo necesita más épocas para converger mejor.

In [15]:
my $num_epochs = 500;
my $lr = 1;
train_ch8($net, $train_iter, $vocab, $lr, $num_epochs, d2l->try_gpu());


Error: Undefined subroutine &RNNModelScratch::train_ch8 called at reply input line 3.


Finalmente, verifiquemos los resultados del uso del método de muestreo aleatorio.

In [16]:
my $net = RNNModelScratch(scalar(@vocab), $num_hiddens, d2l->try_gpu(), $get_params, $init_rnn_state, $rnn);
train_ch8($net, $train_iter, \@vocab, $lr, $num_epochs, d2l->try_gpu(), 1);


Error: Global symbol "@vocab" requires explicit package name (did you forget to declare "my @vocab"?) at reply input line 1.
Global symbol "$get_params" requires explicit package name (did you forget to declare "my $get_params"?) at reply input line 1.
Global symbol "$init_rnn_state" requires explicit package name (did you forget to declare "my $init_rnn_state"?) at reply input line 1.
Global symbol "$rnn" requires explicit package name (did you forget to declare "my $rnn"?) at reply input line 1.
Global symbol "@vocab" requires explicit package name (did you forget to declare "my @vocab"?) at reply input line 2.
BEGIN not safe after errors--compilation aborted at reply input line 7.



Aunque implementar el modelo RNN anterior desde cero es instructivo, no es conveniente. En la siguiente sección, veremos cómo mejorar el modelo RNN, por ejemplo, cómo hacerlo más fácil de implementar y hacer que se ejecute más rápido.

## 8.5.7. Resumen
Podemos entrenar un modelo de lenguaje a nivel de caracteres basado en RNN para generar texto siguiendo el prefijo de texto proporcionado por el usuario.

Un modelo de lenguaje RNN simple consta de codificación de entrada, modelado RNN y generación de salida.

Los modelos RNN necesitan la inicialización del estado para el entrenamiento, aunque el muestreo aleatorio y la partición secuencial usan formas diferentes.

Cuando usamos partición secuencial, necesitamos separar el gradiente para reducir el costo computacional.

Un período de calentamiento permite que un modelo se actualice (p. ej., obtenga un mejor estado oculto que su valor inicializado) antes de realizar cualquier predicción.

El recorte de degradado evita la explosión de degradado, pero no puede corregir los degradados que se desvanecen.

### 8.5.8. Ejercicios
Muestre que la codificación one-hot es equivalente a elegir una incrustación diferente para cada objeto.

Ajuste los hiperparámetros (p. ej., número de épocas, número de unidades ocultas, número de pasos de tiempo en un minilote y tasa de aprendizaje) para mejorar la perplejidad.

¿Cuan bajo Puedes ir?

Reemplace la codificación one-hot con incrustaciones aprendibles. ¿Esto conduce a un mejor rendimiento?

¿Qué tan bien funcionará en otros libros de HG Wells, por ejemplo, La guerra de los mundos ?

Modifique la función de predicción para utilizar el muestreo en lugar de seleccionar el siguiente carácter más probable.

¿Lo que sucede?

Sesgar el modelo hacia resultados más probables, por ejemplo, muestreando de  q(xt∣xt−1,…,x1)∝P(xt∣xt−1,…,x1)α  para α>1 .

Ejecute el código de esta sección sin recortar el degradado. ¿Lo que sucede?

Cambie la partición secuencial para que no separe los estados ocultos del gráfico computacional. ¿Cambia el tiempo de ejecución? ¿Qué tal la perplejidad?

Reemplace la función de activación utilizada en esta sección con ReLU y repita los experimentos en esta sección. ¿Todavía necesitamos recorte de degradado? ¿Por qué?